In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
data.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [4]:
train,test=train_test_split(data,test_size=0.25,random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [5]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8], dtype=int64)

In [6]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()


In [7]:
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

In [8]:

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [9]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [12]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [11]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [13]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/03/07 13:48:07 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\saira\anaconda3\envs\MLLearning\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

Epoch 1/3                                            

  0%|          | 0/4 [00:01<?, ?trial/s, best loss=?]WARNING:tensorflow:From c:\Users\saira\anaconda3\envs\MLLearning\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

46/46 [==============================] - 1s 7ms/step - loss: 15.4170 - root_mean_squared_error: 3.9265 - val_loss: 5.3368 - val_root_mean_squared_error: 2.3102

Epoch 2/3                                            

46/46 [==============================] - 0s 2ms/step - loss: 3.6062 - root_mean_squared_error: 1.8990 - val_loss: 2.6760 - val_root_mean_squared_error: 1

INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpfiw4uhpw\model\data\model\assets



Epoch 1/3                                                                      

46/46 [==============================] - 1s 5ms/step - loss: 36.2003 - root_mean_squared_error: 6.0167 - val_loss: 33.1984 - val_root_mean_squared_error: 5.7618

Epoch 2/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 29.9726 - root_mean_squared_error: 5.4747 - val_loss: 26.7249 - val_root_mean_squared_error: 5.1696

Epoch 3/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 24.1075 - root_mean_squared_error: 4.9099 - val_loss: 21.4085 - val_root_mean_squared_error: 4.6269

12/12 [==============================] - 0s 2ms/step - loss: 21.4085 - root_mean_squared_error: 4.6269

 25%|██▌       | 1/4 [00:18<00:51, 17.19s/trial, best loss: 1.4787137508392334]INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpu99166sr\model\d

INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpu99166sr\model\data\model\assets



...

Epoch 1/3                                                                      

46/46 [==============================] - 1s 5ms/step - loss: 4.8353 - root_mean_squared_error: 2.1989 - val_loss: 1.3609 - val_root_mean_squared_error: 1.1666

Epoch 2/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 1.1097 - root_mean_squared_error: 1.0534 - val_loss: 0.9919 - val_root_mean_squared_error: 0.9959

Epoch 3/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 0.8607 - root_mean_squared_error: 0.9277 - val_loss: 0.8156 - val_root_mean_squared_error: 0.9031

12/12 [==============================] - 0s 1ms/step - loss: 0.8156 - root_mean_squared_error: 0.9031

 75%|███████▌  | 3/4 [00:45<00:14, 14.52s/trial, best loss: 1.4787137508392334]INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpn2tsb8pc\model\data\mod

INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpn2tsb8pc\model\data\model\assets



100%|██████████| 4/4 [00:58<00:00, 14.53s/trial, best loss: 0.9031071066856384]
INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpi9l0_0t3\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\saira\AppData\Local\Temp\tmpi9l0_0t3\model\data\model\assets



Best parameters: {'lr': 0.00662711413542223, 'momentum': 0.5862268809765918}
Best eval rmse: 0.9031071066856384
